In [1]:
!pip install qiskit[visualization]==1.1.0
# Use the following if you are on MacOS/zsh
#!pip install 'qiskit[visualization]'==1.1.0
!pip install qiskit_aer
!pip install qiskit_ibm_runtime
!pip install matplotlib
!pip install pylatexenc
!pip install prototype-zne
!pip install git+https://github.com/qiskit-community/Quantum-Challenge-Grader.git

  Using cached qiskit-1.1.0-cp38-abi3-win_amd64.whl.metadata (13 kB)
Using cached qiskit-1.1.0-cp38-abi3-win_amd64.whl (4.2 MB)
  Attempting uninstall: qiskit
    Found existing installation: qiskit 1.0.2
    Uninstalling qiskit-1.0.2:
      Successfully uninstalled qiskit-1.0.2
  Cloning https://github.com/qiskit-community/Quantum-Challenge-Grader.git to c:\users\admin\appdata\local\temp\pip-req-build-b2rcsdi_
  Resolved https://github.com/qiskit-community/Quantum-Challenge-Grader.git to commit bcaff898473fd54f3e2668f162817947787f7268
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/qiskit-community/Quantum-Challenge-Grader.git 'C:\Users\Admin\AppData\Local\Temp\pip-req-build-b2rcsdi_'


In [36]:
%set_env QXToken=YOUR_TOKEN

env: QXToken=YOUR_TOKEN


In [3]:
from qc_grader.challenges.qgss_2024 import *

from math import pi
from qiskit.circuit.library import QFT
from qiskit.providers.fake_provider import GenericBackendV2, generic_backend_v2
generic_backend_v2._NOISE_DEFAULTS["cx"] = (5.99988e-06, 6.99988e-06, 1e-5, 5e-3)

from qiskit import transpile, QuantumCircuit
from qiskit.circuit import Gate
from qiskit.converters import circuit_to_dag
from qiskit.transpiler import CouplingMap, StagedPassManager, PassManager, AnalysisPass, TransformationPass
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.transpiler.preset_passmanagers.common import generate_unroll_3q, generate_embed_passmanager
from qiskit.quantum_info import hellinger_fidelity
from qiskit.providers.basic_provider import BasicSimulator
from qiskit.dagcircuit import DAGCircuit
from qiskit_ibm_runtime.fake_provider import FakeTorino

# Transpiler Passes
## Layout passes
from qiskit.transpiler.passes.layout.csp_layout import CSPLayout
from qiskit.transpiler.passes.layout.dense_layout import DenseLayout
from qiskit.transpiler.passes.layout.sabre_layout import SabreLayout
from qiskit.transpiler.passes.layout.vf2_layout import VF2Layout
from qiskit.transpiler.passes.layout.trivial_layout import TrivialLayout

## Routing passes
from qiskit.transpiler.passes.routing.basic_swap import BasicSwap
from qiskit.transpiler.passes.routing.lookahead_swap import LookaheadSwap
from qiskit.transpiler.passes.routing.sabre_swap import SabreSwap
from qiskit.transpiler.passes.routing.stochastic_swap import StochasticSwap
from qiskit.transpiler.passes.routing.star_prerouting import StarPreRouting

## Synthesis passes (passes for the translation stage)
from qiskit.circuit import SessionEquivalenceLibrary
from qiskit.circuit.equivalence_library import SessionEquivalenceLibrary
from qiskit.transpiler.passes.basis.basis_translator import BasisTranslator
from qiskit.transpiler.passes.synthesis.high_level_synthesis import HighLevelSynthesis
### The next pass could also be considered an optimization pass.
from qiskit.transpiler.passes.synthesis.unitary_synthesis import UnitarySynthesis

## Optimization passes
from qiskit.transpiler.passes.optimization.collect_1q_runs import Collect1qRuns
from qiskit.transpiler.passes.optimization.collect_2q_blocks import Collect2qBlocks
from qiskit.transpiler.passes.optimization.consolidate_blocks import ConsolidateBlocks
from qiskit.transpiler.passes.optimization.commutative_cancellation import CommutativeCancellation

In [4]:
# get an abstract quantum circuit from Qiskit's library of circuits
num_qubits = 10
qc = QFT(num_qubits, do_swaps=False)
qc.draw()

┌──────┐
q_0: ┤0     ├
     │      │
q_1: ┤1     ├
     │      │
q_2: ┤2     ├
     │      │
q_3: ┤3     ├
     │      │
q_4: ┤4     ├
     │  QFT │
q_5: ┤5     ├
     │      │
q_6: ┤6     ├
     │      │
q_7: ┤7     ├
     │      │
q_8: ┤8     ├
     │      │
q_9: ┤9     ├
     └──────┘

# 1

In [6]:
from qiskit import QuantumCircuit

def get_qc_characteristics(qc):
    # determine the quantum circuit depth of `qc` and assign it to `depth`
    depth = qc.depth()
    
    # determine the number of qubits in `qc` and assign it to `num_qubits`
    num_qubits = qc.num_qubits
    
    # determine the operations in `qc` and assign it to `ops`
    ops = qc.count_ops()
    
    # determine the number of n-qubit operations (with n larger than 1) in `qc` and assign it to `num_multi_qubit_ops`
    num_multi_qubit_ops =  sum(1 for instr, qargs, _ in qc.data if len(qargs) > 1)
    #num_multi_qubit_ops = qc.num_nonlocal_gates()
    
    # do not modify the next line
    return {"depth": depth, "num_qubits": num_qubits, "ops": ops, "num_multi_qubit_ops": num_multi_qubit_ops}

# Exemplo de uso:
qc = QFT(num_qubits, do_swaps=False)
characteristics = get_qc_characteristics(qc)
print(characteristics)

{'depth': 1, 'num_qubits': 10, 'ops': OrderedDict({'QFT': 1}), 'num_multi_qubit_ops': 1}


In [7]:
# Grade your work!
grade_lab1_ex1(get_qc_characteristics)

Submitting your answer. Please wait...
Congratulations 🎉! Your answer is correct and has been submitted.


In [8]:
# print quantum circuit characteristics
def print_qc_characteristics(qc):
    characteristics = get_qc_characteristics(qc)
    print("Quantum circuit characteristics")
    print("  Depth:", characteristics['depth'])
    print("  Number of qubits:", characteristics['num_qubits'])
    print("  Operations:", dict(characteristics['ops']))
    print("  Number of multi-qubit Operations:", characteristics['num_multi_qubit_ops'])

print_qc_characteristics(qc)

Quantum circuit characteristics
  Depth: 1
  Number of qubits: 10
  Operations: {'QFT': 1}
  Number of multi-qubit Operations: 1


# 2

In [9]:
qc_dec = qc.decompose()
get_qc_characteristics(qc_dec)
qc_dec.draw(fold=-1)

┌───┐
q_0: ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■────────────────────────────────────────────■──────────────────────────────────────■──────────────────────────────────■───────────────────────────■───────────────────────■──────────────────■─────────────■────────■───────┤ H ├
                                                                                                                                                                                                                                                                     │                                            │                                      │                                  │                           │                       │                  │       ┌───┐ │        │P(π/2) └───┘
q_1: ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■──────────────────────────────────────■──────────┼──────────────────────────────────■─────────┼────────────────────────────■─────────┼────────────────────────■─────────┼──────────────────■────────┼──────────────■────────┼─────────■────────┼───────┤ H ├─┼────────■────────────
                                                                                                                                                                                                                   │                                      │          │                                  │         │                            │         │                        │         │                  │        │        ┌───┐ │        │         │P(π/2)  │       └───┘ │P(π/4)               
q_2: ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■──────────────────────────────────■─────────┼────────────────────────────■─────────┼──────────┼────────────────────────■─────────┼─────────┼───────────────────■────────┼─────────┼───────────────■────────┼─────────┼─────────■────────┼────────┼────────┤ H ├─┼────────┼─────────■────────┼─────────────■─────────────────────
                                                                                                                                                                      │                                  │         │                            │         │          │                        │         │         │                   │        │         │         ┌───┐ │        │         │         │P(π/2)  │        │        └───┘ │P(π/4)  │                  │P(π/8)                             
q_3: ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■───────────────────────────■─────────┼────────────────────────■─────────┼─────────┼───────────────────■────────┼─────────┼──────────┼───────────────■────────┼─────────┼─────────┼──────────■────────┼────────┼─────────┼─────────┤ H ├─┼────────┼─────────┼─────────■────────┼────────┼──────────────■────────┼──────────────────■───────────────────────────────────
                                                                                                                                │                           │         │                        │         │         │                   │        │         │          │         ┌───┐ │        │         │         │          │P(π/2)  │        │         │         └───┘ │P(π/4)  │         │                  │P(π/8)  │                       │P(π/16)                                               
q_4: ─────────────────────────────────────────────────────────────────────────────────────

In [10]:
backend = GenericBackendV2(num_qubits)
print("Supported basis gates:", backend.operation_names)

Supported basis gates: ['cx', 'id', 'rz', 'sx', 'x', 'reset', 'delay', 'measure']


In [11]:
# transpile the qft quantum circuit for the basis gate of our example backend
qc_synth = generate_preset_pass_manager(2, backend=backend).run(qc)
qc_synth.draw(fold=-1)

global phase: 0
                                                                                                                                                                                                                                                                                                                                   ┌───┐┌────────────┐                    ┌───┐   ┌───────────┐                                   ┌───┐     ┌────────────┐        ┌───┐    ┌───────────┐                  ┌───┐    ┌───────────┐                  ┌───┐    ┌──────────┐                              ┌───┐    ┌───────────┐            ┌───┐┌──────────┐                                          ┌───┐┌───────────┐                               ┌───┐┌──────────┐┌───┐ ┌──────────┐┌───┐  ┌─────────┐                   ┌───┐    ┌──────────┐   ┌───┐    ┌──────────┐    ┌────┐   ┌──────────┐                                                                                                                                                                                       
q_2 -> 0 ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤ X ├┤ Rz(-π/256) ├────────────────────┤ X ├───┤ Rz(π/256) ├───────────────────────────────────┤ X ├─────┤ Rz(-π/128) ├────────┤ X ├────┤ Rz(π/128) ├──────────────────┤ X ├────┤ Rz(-π/64) ├──────────────────┤ X ├────┤ Rz(π/64) ├──────────────────────────────┤ X ├────┤ Rz(-π/32) ├────────────┤ X ├┤ Rz(π/32) ├──────────────────────────────────────────┤ X ├┤ Rz(-π/16) ├───────────────────────────────┤ X ├┤ Rz(π/16) ├┤ X ├─┤ Rz(-π/8) ├┤ X ├──┤ Rz(π/8) ├───────────────────┤ X ├────┤ Rz(-π/4) ├───┤ X ├────┤ Rz(3π/4) ├────┤ √X ├───┤ Rz(7π/8) ├───────────────────────────────────────■────────────────────────────■────────────────────■────────────────────────────■────────────────────────────────────────────────────────────────
                                                                                                                                                                                                                         ┌───┐┌───────────┐             ┌───┐┌──────────┐                 ┌───┐    ┌───────────┐                   └─┬─┘└───┬───┬────┘┌──────────┐        └─┬─┘   └───┬───┬───┘┌───────────┐         ┌───┐        └─┬─┘     └┬──────────┬┘        └─┬─┘    └───┬───┬───┘  ┌──────────┐    └─┬─┘    └───┬───┬───┘ ┌─────────┐      └─┬─┘    └──┬───┬───┘ ┌──────────┐                 └─┬─┘    └───┬───┬───┘┌──────────┐└─┬─┘└──┬────┬──┘┌────────────┐                            └─┬─┘└───────────┘                               └─┬─┘└──────────┘└─┬─┘ └──────────┘└─┬─┘  └─────────┘                   └─┬─┘    └──────────┘   └─┬─┘    └──────────┘    └────┘   └──────────┘                                       │                            │                    │                            │                                                                
q_4 -> 1 ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤ X ├┤ Rz(-π/64) ├─────────────┤ X ├┤ Rz(π/64) ├─────────────────┤ X ├────┤ Rz(-π/32) ├─────────────────────┼──────┤ X ├─────┤ Rz(π/32) ├──────────┼─────────┤ X ├────┤ Rz(-π/16) ├─────────┤ X ├──────────┼────────┤ Rz(π/16) ├───────────┼──────────┤ X ├──────┤ Rz(-π/8) ├──────┼──────────┤ X ├─────┤ Rz(π/8) ├────────┼─────────┤ X ├─────┤ Rz(-π/4) ├───────────────────┼──────────┤ X ├────┤ Rz(3π/4) ├──┼─────┤ √X ├───┤ Rz(3.0434) ├──────────────────────────────┼────────────────────■───────────────────────────┼───────■────────■─────────────────■───────────────────────■────────────┼───────────■───────────┼───────────■───

In [12]:
print_qc_characteristics(qc_synth)

Quantum circuit characteristics
  Depth: 65
  Number of qubits: 10
  Operations: {'rz': 101, 'cx': 90, 'sx': 10}
  Number of multi-qubit Operations: 90


In [13]:
# assign a 10-qubit linear CouplingMap to the variable cm
from qiskit.transpiler import CouplingMap

#cm  = [[0, 1], [1, 2], [2, 3], [3, 4], [4, 5], [5, 6], [6, 7], [7, 8], [8, 9]] 
cm = CouplingMap([[i, i+1] for i in range(num_qubits - 1)])

# add your transpiled quantum circuit to the next line
qc_routed = transpile(qc,coupling_map=cm, backend= backend)


In [14]:
# grade your work!
grade_lab1_ex2(qc_routed)

Submitting your answer. Please wait...
Congratulations 🎉! Your answer is correct and has been submitted.


# 3

In [15]:
# Your work goes here!

pm_staged = StagedPassManager()
# Select pass managers and passes for these five stages as you see suitable, note: not every stage must contain a pass.
# Passes are documented at https://docs.quantum.ibm.com/api/qiskit/transpiler_passes

# Replace the n-qubit QFT operation with its decomposition in two-qubit gates
pm_staged.init = generate_unroll_3q(None)
# Initialize the layout stage with an empty pass manager
pm_staged.layout = PassManager()
# See the first cells in this notebook or
#   https://github.com/Qiskit/qiskit/tree/main/qiskit/transpiler/passes/layout for potential layouting passes
pm_staged.layout += TrivialLayout(cm)
# See the first cells in this notebook or
#   https://github.com/Qiskit/qiskit/tree/main/qiskit/transpiler/passes/routing for potential routing passes
pm_staged.routing = PassManager(StarPreRouting())
# See the first cells in this notebook or
#   https://github.com/Qiskit/qiskit/tree/main/qiskit/transpiler/passes/synthesis for potential translation passes
pm_staged.translation = PassManager(
    [
        Collect2qBlocks(),
        ConsolidateBlocks(basis_gates=backend.operation_names),
        UnitarySynthesis(basis_gates=backend.operation_names)
    ]
)
# See the first cells in this notebook or
#   https://github.com/Qiskit/qiskit/tree/main/qiskit/transpiler/passes/optimization for potential optimization passes
pm_staged.optimization = None
# scheduling is not needed in this lab - we will not run the quantum circuit on a real device
# pm_opt.scheduling = None

# do not modify the next line
pm_staged.layout += generate_embed_passmanager(cm)

qc_staged = pm_opt.run(qc)

NameError: name 'pm_opt' is not defined

In [16]:
qc_routed = pm_staged.run(qc)
print_qc_characteristics(qc_routed)
qc_routed.draw(fold=-1)

Quantum circuit characteristics
  Depth: 178
  Number of qubits: 10
  Operations: {'rz': 371, 'sx': 186, 'cx': 133, 'x': 38}
  Number of multi-qubit Operations: 133


global phase: 4.7139
           ┌────────┐      ┌────┐                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             ┌───┐    ┌─────────┐                  ┌───┐    ┌────────────┐    ┌───┐     ┌────────────┐                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
q_0 -> 0 ──┤ Rz(-π) ├──────┤ √X ├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤ X ├────┤ Rz(π/2) ├──────────────────┤ X ├────┤ Rz(1.5677) ├────┤ X ├─────┤ Rz(π/1024) ├──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [17]:
# grade your work!
grade_lab1_ex3(pm_staged)

Submitting your answer. Please wait...
Congratulations 🎉! Your answer is correct and has been submitted.


# 4

In [18]:
pm_staged.translation = PassManager(BasisTranslator(equivalence_library=SessionEquivalenceLibrary, target_basis=backend.operation_names))
qc_routed_synth = pm_staged.run(qc)
print_qc_characteristics(qc_routed_synth)
qc_routed_synth.draw(fold=-1)

Quantum circuit characteristics
  Depth: 127
  Number of qubits: 10
  Operations: {'cx': 219, 'rz': 155, 'sx': 10}
  Number of multi-qubit Operations: 219


global phase: 1.5723
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          ┌───┐    ┌─────────────┐    ┌───┐     ┌────────────┐    ┌───┐                   ┌───┐                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
q_0 -> 0 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤ X ├────┤ Rz(-π/1024) ├────┤ X ├─────┤ Rz(π/1024) ├────┤ X ├─────────■─────────┤ X ├───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  ┌───┐    ┌────────────┐    ┌───┐     ┌───────────┐    ┌───┐                   ┌───┐   ┌────────────┐    └─┬─┘    └─────────────┘    └─┬─┘     └────────────┘    └─┬─┘       ┌─┴─┐       └─┬─┘                          ┌───┐    ┌────────────┐    ┌───┐     ┌───────────┐    ┌───┐                   ┌───┐                                                                                                                                 

In [19]:
# grade your work!
grade_lab1_ex4(pm_staged)

Submitting your answer. Please wait...
Congratulations 🎉! Your answer is correct and has been submitted.


# 5

In [20]:
qk_qc = generate_preset_pass_manager(2, backend=backend).run(qc)
print_qc_characteristics(qk_qc)

Quantum circuit characteristics
  Depth: 65
  Number of qubits: 10
  Operations: {'rz': 101, 'cx': 90, 'sx': 10}
  Number of multi-qubit Operations: 90


In [21]:
def noisy_sim(qc, backend):
    # We add measurement operations to the input quantum circuit and then run it on the specified backend
    # A GenericBackendV2 automatically constructs a default model of the expected noise processes,
    # so backend.run would return noisy simulation results
    return backend.run(qc.measure_all(inplace=False), shots=7 * 1024).result().get_counts()

own_transpiler_sim = noisy_sim(qc_routed_synth, backend)
qiskit_transpiler_sim = noisy_sim(qk_qc, backend)
reference_sim = noisy_sim(transpile(qc.decompose(), backend=backend), BasicSimulator())

print("Own transpiler fidelity", round(hellinger_fidelity(own_transpiler_sim, reference_sim), 4))
print("Qiskit transpiler fidelity", round(hellinger_fidelity(qiskit_transpiler_sim, reference_sim), 4))

Own transpiler fidelity 0.2431
Qiskit transpiler fidelity 0.5879


In [22]:
# Your work goes here!

pm_opt = StagedPassManager()
# Select pass managers and passes for these five stages as you see suitable, note: not every stage must contain a pass.
# Passes are documented at https://docs.quantum.ibm.com/api/qiskit/transpiler_passes

# Replace the n-qubit QFT operation with its decomposition in two-qubit gates
pm_opt.init = generate_unroll_3q(None)
# Initialize the layout stage with an empty pass manager
pm_opt.layout = PassManager()
# See the first cells in this notebook or
#   https://github.com/Qiskit/qiskit/tree/main/qiskit/transpiler/passes/layout for potential layouting passes
pm_opt.layout += TrivialLayout(cm)
# See the first cells in this notebook or
#   https://github.com/Qiskit/qiskit/tree/main/qiskit/transpiler/passes/routing for potential routing passes
pm_opt.routing = PassManager(StarPreRouting())
# See the first cells in this notebook or
#   https://github.com/Qiskit/qiskit/tree/main/qiskit/transpiler/passes/synthesis for potential translation passes
pm_opt.translation = PassManager(
    [
        Collect2qBlocks(),
        ConsolidateBlocks(basis_gates=backend.operation_names),
        UnitarySynthesis(basis_gates=backend.operation_names)
    ]
)
# See the first cells in this notebook or
#   https://github.com/Qiskit/qiskit/tree/main/qiskit/transpiler/passes/optimization for potential optimization passes
pm_opt.optimization = None
# scheduling is not needed in this lab - we will not run the quantum circuit on a real device
# pm_opt.scheduling = None

# do not modify the next line
pm_opt.layout += generate_embed_passmanager(cm)

qc_opt = pm_opt.run(qc)

In [23]:
# grade your work!
grade_lab1_ex5(pm_opt)

Submitting your answer. Please wait...
Congratulations 🎉! Your answer is correct and has been submitted.


# 6

In [25]:
print_qc_characteristics(qc_opt)
reduction_ratio = round(100-100*(get_qc_characteristics(qc_opt)['num_multi_qubit_ops']/get_qc_characteristics(qk_qc)['num_multi_qubit_ops']), 3)
print("Reduction in two-qubit gates compared to qiskit {}%!".format(reduction_ratio))
qc_opt.draw(fold=-1)

Quantum circuit characteristics
  Depth: 178
  Number of qubits: 10
  Operations: {'rz': 371, 'sx': 186, 'cx': 133, 'x': 38}
  Number of multi-qubit Operations: 133
Reduction in two-qubit gates compared to qiskit -47.778%!


global phase: 4.7139
           ┌────────┐      ┌────┐                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             ┌───┐    ┌─────────┐                  ┌───┐    ┌────────────┐    ┌───┐     ┌────────────┐                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
q_0 -> 0 ──┤ Rz(-π) ├──────┤ √X ├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤ X ├────┤ Rz(π/2) ├──────────────────┤ X ├────┤ Rz(1.5677) ├────┤ X ├─────┤ Rz(π/1024) ├──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [26]:
opt_transpiler_sim = noisy_sim(qc_opt, backend)

print("Own transpiler fidelity", round(hellinger_fidelity(own_transpiler_sim, reference_sim), 4))
print("Qiskit transpiler fidelity", round(hellinger_fidelity(qiskit_transpiler_sim, reference_sim), 4))
print("Own optimized transpiler fidelity", round(hellinger_fidelity(opt_transpiler_sim, reference_sim), 4))

Own transpiler fidelity 0.2431
Qiskit transpiler fidelity 0.5879
Own optimized transpiler fidelity 0.8995


In [27]:
from qiskit.passmanager import PropertySet
myprop=PropertySet()
class GatesPerQubit(AnalysisPass):
    # Your work here - make sure you implement every abstract method defined in AnalysisPass (see https://docs.quantum.ibm.com/api/qiskit/qiskit.transpiler.AnalysisPass)

    def run(self, dag: DAGCircuit):  # pylint: disable=arguments-differ
        def get_one_q_ops_dict(dag):
            one_q_ops_dict = {}
            for alist in dag.collect_1q_runs():
                dagnode = alist[0]
                qbit_obj = dagnode.qargs[0]
                if str(qbit_obj._index) in one_q_ops_dict.keys():
                    one_q_ops_dict[qbit_obj] += len(alist)
                else:
                    one_q_ops_dict[qbit_obj] = len(alist)
            return one_q_ops_dict

        def get_two_q_ops_dict(dag):
            two_q_ops_dict = {}
            for alist in dag.collect_2q_runs():
                dagnode = alist[0]
                qbit_obj = dagnode.qargs[0]
                if str(qbit_obj._index) in two_q_ops_dict.keys():
                    # two_q_ops_dict[str(qbit_obj._index)] += len(alist)
                    two_q_ops_dict[qbit_obj] += len(alist)
                else:
                    two_q_ops_dict[qbit_obj] = len(alist)
            return two_q_ops_dict

        # property_set = PropertySet()

        one_q_ops_dict = get_one_q_ops_dict(dag)
        self.property_set['one_q_op'] = one_q_ops_dict

        two_q_ops_dict = get_two_q_ops_dict(dag)
        self.property_set['two_q_op'] = two_q_ops_dict
    pass

In [28]:
from qiskit.transpiler.basepasses import AnalysisPass
from qiskit.transpiler import PassManager
from qiskit import QuantumCircuit

class GatesPerQubit(AnalysisPass):
    def __init__(self):
        super().__init__()

    def run(self, dag):
        # Inicializa os dicionários para contar as operações de um e dois qubits
        one_q_op = {qubit: 0 for qubit in dag.qubits}
        two_q_op = {qubit: 0 for qubit in dag.qubits}

        # Colete os nomes das operações de um e dois qubits
        names_ops1 = self.collectNamesOps(dag, 1)
        names_ops2 = self.collectNamesOps(dag, 2)

        # Contar as operações de um qubit
        for name_op_1 in names_ops1:
            for node in dag.op_nodes():
                op_instruction = node.op
                op_qubits = node.qargs
                if op_instruction.name == name_op_1:
                    for q in op_qubits:
                        one_q_op[q] += 1

        # Contar as operações de dois qubits
        for name_op_2 in names_ops2:
            for node in dag.op_nodes():
                op_instruction = node.op
                op_qubits = node.qargs
                if op_instruction.name == name_op_2:
                    for q in op_qubits:
                        two_q_op[q] += 1

        # Armazenar os resultados em property_set
        self.property_set["one_q_op"] = one_q_op
        self.property_set["two_q_op"] = two_q_op

    def collectNamesOps(self, dag, num_qubits):
        # Esta função coleta os nomes das operações de um ou dois qubits
        ops = set()
        for node in dag.op_nodes():
            if len(node.qargs) == num_qubits:
                ops.add(node.op.name)
        return ops

# Exemplo de uso:
qc = QuantumCircuit(3)
qc.h(0)
qc.cx(0, 1)
qc.cx(1, 2)
qc.measure_all()

pass_manager = PassManager(GatesPerQubit())
pass_manager.run(qc)

# Acessando os resultados armazenados
property_set = pass_manager.property_set
print("Single-qubit gate counts:", property_set["one_q_op"])
print("Two-qubit gate counts:", property_set["two_q_op"])


Single-qubit gate counts: {Qubit(QuantumRegister(3, 'q'), 0): 2, Qubit(QuantumRegister(3, 'q'), 1): 1, Qubit(QuantumRegister(3, 'q'), 2): 1}
Two-qubit gate counts: {Qubit(QuantumRegister(3, 'q'), 0): 1, Qubit(QuantumRegister(3, 'q'), 1): 2, Qubit(QuantumRegister(3, 'q'), 2): 1}


In [29]:
# grade your work!
grade_lab1_ex6(GatesPerQubit)

Submitting your answer. Please wait...
Congratulations 🎉! Your answer is correct and has been submitted.


# 7

In [30]:
pg = Gate('Peres', 3, params=[], label='PG')

In [31]:
qc_pg = QuantumCircuit(3)
qc_pg.append(pg, [0, 1, 2])
qc_pg.draw()

┌─────┐
q_0: ┤0    ├
     │     │
q_1: ┤1 PG ├
     │     │
q_2: ┤2    ├
     └─────┘

In [32]:
def get_qc_in(nq):
    # QFT circuit, feel free to use a previously defined pass manager for the QFT circuit
    qc_qft = QFT(nq, do_swaps=False)
    # part of the circuit including the Peres gate
    qc_inner = QuantumCircuit(nq)
    for i in range(1, nq - 1):
        qc_inner.append(pg, [nq - i - 2, nq - i - 1, nq - 1])

    qc_in = QuantumCircuit(nq)
    # add QFT circuit to qc_in
    qc_in.compose(qc_qft, range(nq), inplace=True)

    # perform swap gates
    for i in range(nq // 2):
        qc_in.cx(i, nq - i - 1)
        qc_in.cx(nq - i - 1, i)
        qc_in.cx(i, nq - i - 1)

    qc_in.rz(pi, nq - 1)
    # add circuit with peres gates
    qc_in.compose(qc_inner, range(nq), inplace=True)

    # perform swap gates
    for i in range(nq // 2):
        qc_in.cx(i, nq - i - 1)
        qc_in.cx(nq - i - 1, i)
        qc_in.cx(i, nq - i - 1)
    # add inverse QFT circuit
    qc_in.compose(qc_qft.inverse(), range(nq), inplace=True)
    return qc_in

nq = 5
qc_in = get_qc_in(nq)
qc_in.draw(fold=-1)

┌──────┐          ┌───┐                                 ┌─────┐          ┌───┐               ┌───────┐
q_0: ┤0     ├──■───────┤ X ├───────■─────────────────────────┤0    ├──■───────┤ X ├───────■───────┤0      ├
     │      │  │       └─┬─┘┌───┐  │                  ┌─────┐│     │  │       └─┬─┘┌───┐  │       │       │
q_1: ┤1     ├──┼────■────┼──┤ X ├──┼──────■───────────┤0    ├┤1    ├──┼────■────┼──┤ X ├──┼────■──┤1      ├
     │      │  │    │    │  └─┬─┘  │      │    ┌─────┐│     ││     │  │    │    │  └─┬─┘  │    │  │       │
q_2: ┤2 QFT ├──┼────┼────┼────┼────┼──────┼────┤0    ├┤1    ├┤  PG ├──┼────┼────┼────┼────┼────┼──┤2 IQFT ├
     │      │  │  ┌─┴─┐  │    │    │    ┌─┴─┐  │     ││  PG ││     │  │  ┌─┴─┐  │    │    │  ┌─┴─┐│       │
q_3: ┤3     ├──┼──┤ X ├──┼────■────┼────┤ X ├──┤1 PG ├┤     ├┤     ├──┼──┤ X ├──┼────■────┼──┤ X ├┤3      ├
     │      │┌─┴─┐└───┘  │       ┌─┴─┐┌─┴───┴─┐│     ││     ││     │┌─┴─┐└───┘  │       ┌─┴─┐└───┘│       │
q_4: ┤4     ├┤ X ├───────■───────┤ X ├┤ Rz(π) ├┤2    ├┤2    ├┤2    ├┤ X ├───────■───────┤ X ├─────┤4      ├
     └──────┘└───┘               └───┘└───────┘└─────┘└─────┘└─────┘└───┘               └───┘     └───────┘

In [33]:
class PeresGateTranslation(TransformationPass):
    def get_peres_decomposition(self):
        qcsx = QuantumCircuit(2)
        qcsx.rz(pi / 4, 0)
        qcsx.rz(pi / 2, 1)
        qcsx.sx(1)
        qcsx.rz(pi / 2, 1)
        qcsx.cx(0, 1)
        qcsx.rz(-pi / 4, 1)
        qcsx.cx(0, 1)
        qcsx.rz(3 * pi / 4, 1)
        qcsx.sx(1)
        qcsx.rz(pi / 2, 1)

        qcsx_inv = QuantumCircuit(2)
        qcsx_inv.rz(pi / 4, 1)
        qcsx_inv.cx(0, 1)
        qcsx_inv.rz(-pi / 4, 1)
        qcsx_inv.cx(0, 1)
        qcsx_inv.rz(pi / 2, 0)
        qcsx_inv.rz(pi / 2, 1)
        qcsx_inv.cx(0, 1)
        qcsx_inv.rz(pi / 2, 1)
        qcsx_inv.sx(1)
        qcsx_inv.rz(-3 * pi / 4, 1)
        qcsx_inv.sx(1)
        qcsx_inv.cx(0, 1)
        qcsx_inv.sx(1)
        qcsx_inv.rz(-3 * pi / 4, 1)
        qcsx_inv.sx(1)
        qcsx_inv.rz(-3 * pi / 4, 1)
        qcsx_inv.cx(0, 1)
        qcsx_inv.rz(-pi / 4, 1)
        qcsx_inv.cx(0, 1)
        qcsx_inv.rz(pi / 4, 0)

        qc_dec = QuantumCircuit(3)
        qc_dec.cx(0, 1)
        qc_dec.cx(1, 0)
        qc_dec.cx(0, 1)
        qc_dec.compose(qcsx, [1, 2], inplace=True)
        qc_dec.cx(0, 1)
        qc_dec.cx(1, 0)
        qc_dec.cx(0, 1)
        qc_dec.compose(qcsx, [1, 2], inplace=True)
        qc_dec.cx(0, 1)
        qc_dec.compose(qcsx_inv, [1, 2], inplace=True)
        qc_dec.cx(0, 1)
        qc_dec.cx(0, 1)
        return qc_dec
    def __init__(self):
        """
        Args:
        peres_gate_to_translate: A list of peres gate to translate
        """
    
        self.peres_gate_to_translate = [pg]
        print(self.peres_gate_to_translate)
    # Your work here - make sure you implement every abstract method defined by https://docs.quantum.ibm.com/api/qiskit/qiskit.transpiler.TransformationPass!
    #def __init__(self, peres_gate_to_translate: Optional[Iterable[Gate]] = None):
    #    """
    #    Args:
    #        peres_gate_to_translate: A list of Peres gates to translate.
    #    """
        super().__init__()
        #self.peres_gate_to_translate = peres_gate_to_translate or []
    
    def run(self, dag: DAGCircuit) -> DAGCircuit:
        if self.peres_gate_to_translate is not None:
            peres_gate_list = list(self.peres_gate_to_translate)
    
            for node in dag.op_nodes():
                for gate in peres_gate_list:
                    if gate.name == node.op.name:
                        dag.substitute_node_with_dag(
                            node, circuit_to_dag(self.get_peres_decomposition())
                        )
        print(dag.op_nodes())
        return dag


In [34]:
peres_trans = PeresGateTranslation()
#trans_dag = peres_trans.run(circuit_to_dag(qc_in))
#trans_dag.draw()

[Instruction(name='Peres', num_qubits=3, num_clbits=0, params=[])]


In [35]:
# grade your work!
grade_lab1_ex7(PeresGateTranslation, pg)

[Instruction(name='Peres', num_qubits=3, num_clbits=0, params=[])]
[DAGOpNode(op=Instruction(name='rz', num_qubits=1, num_clbits=0, params=[1.5707963267948966]), qargs=(Qubit(QuantumRegister(5, 'q'), 3),), cargs=()), DAGOpNode(op=Instruction(name='sx', num_qubits=1, num_clbits=0, params=[]), qargs=(Qubit(QuantumRegister(5, 'q'), 3),), cargs=()), DAGOpNode(op=Instruction(name='rz', num_qubits=1, num_clbits=0, params=[2.356194490192345]), qargs=(Qubit(QuantumRegister(5, 'q'), 3),), cargs=()), DAGOpNode(op=Instruction(name='cx', num_qubits=2, num_clbits=0, params=[]), qargs=(Qubit(QuantumRegister(5, 'q'), 3), Qubit(QuantumRegister(5, 'q'), 2)), cargs=()), DAGOpNode(op=Instruction(name='rz', num_qubits=1, num_clbits=0, params=[-0.7853981633974483]), qargs=(Qubit(QuantumRegister(5, 'q'), 2),), cargs=()), DAGOpNode(op=Instruction(name='cx', num_qubits=2, num_clbits=0, params=[]), qargs=(Qubit(QuantumRegister(5, 'q'), 3), Qubit(QuantumRegister(5, 'q'), 2)), cargs=()), DAGOpNode(op=Instruction